<a href="https://colab.research.google.com/github/syh0397/Charter-Dashboard-for-Newbies/blob/main/%ED%8E%B8%EC%9D%98%EC%8B%9C%EC%84%A4_%EC%9C%84%EC%B9%98_%EC%A0%95%EB%B3%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IMPORT AND API CONTROLLER

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json
import requests
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/DATA/비저블 맵 프로젝트/전월세_거리당갯수.csv')
df.reset_index()
df

,Unnamed: 0,자치구명,법정동명,지번주소,건물명,층,임대건물명,임대면적,전월세구분,보증금,...,latitude,longitude,conv_num,in_din_total_300,in_mart_total_1000,in_subway_total_500,in_hosp_total_1000,in_cafe_total_300,police_500,center_500
0,0,영등포구,양평동6가,서울특별시 영등포구 양평동6가 93-2,선유도디와이파크,4,오피스텔,19.52,준전세,11000,...,37.543427,126.888418,3,10,2,1,29,4,1,1
1,1,영등포구,양평동6가,서울특별시 영등포구 양평동6가 93-2,선유도디와이파크,6,오피스텔,18.92,전세,13125,...,37.543427,126.888418,3,10,2,1,29,4,1,1
2,2,영등포구,양평동6가,서울특별시 영등포구 양평동6가 93-2,선유도디와이파크,5,오피스텔,18.92,전세,12600,...,37.543427,126.888418,3,10,2,1,29,4,1,1
3,3,영등포구,양평동6가,서울특별시 영등포구 양평동6가 93-2,선유도디와이파크,3,오피스텔,18.92,전세,9923,...,37.543427,126.888418,3,10,2,1,29,4,1,1
4,4,영등포구,양평동6가,서울특별시 영등포구 양평동6가 84,유성빌라(84-0),2,다세대/연립,50.82,준월세,1000,...,37.543388,126.889033,3,11,2,0,28,6,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1186,1186,서초구,우면동,서울특별시 서초구 우면동 765,미라쉘,3,오피스텔,19.40,준월세,3000,...,37.456478,127.015923,2,8,1,0,6,3,1,0
1187,1187,서초구,우면동,서울특별시 서초구 우면동 764,계룡리슈빌S,7,아파트,21.59,준월세,1000,...,37.456461,127.016237,2,8,1,0,6,3,1,0
1188,1188,서초구,우면동,서울특별시 서초구 우면동 764,계룡리슈빌S,7,아파트,21.59,준월세,10000,...,37.456461,127.016237,2,8,1,0,6,3,1,0
1189,1189,서초구,신원동,서울특별시 서초구 신원동 619,서초포레스타6단지,4,아파트,49.34,전세,13067,...,37.449935,127.055830,6,16,2,1,14,5,0,2


In [3]:
class KakaoLocalAPI:
    """
    Kakao Local API 컨트롤러
    """

    def __init__(self, rest_api_key):
        """
        Rest API키 초기화 및 기능 별 URL 설정
        """

        # REST API 키 설정
        self.rest_api_key = '650c8233432728848e448a6d76f3f155'
        self.headers = {"Authorization": "KakaoAK {}".format(rest_api_key)}

        # 서비스 별 URL 설정

        # 01 주소 검색
        self.URL_01 = "https://dapi.kakao.com/v2/local/search/address.json"
        # 02 좌표-행정구역정보 변환
        self.URL_02 = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json"
        # 03 좌표-주소 변환
        self.URL_03 = "https://dapi.kakao.com/v2/local/geo/coord2address.json"
        # 04 좌표계 변환
        self.URL_04 = "https://dapi.kakao.com/v2/local/geo/transcoord.json"
        # 05 키워드 검색
        self.URL_05 = "https://dapi.kakao.com/v2/local/search/keyword.json"
        # 06 카테고리 검색
        self.URL_06 = "https://dapi.kakao.com/v2/local/search/category.json"

    def search_address(self, query, analyze_type=None, page=None, size=None):
        """
        01 주소 검색
        """
        params = {"query": f"{query}"}

        if analyze_type != None:
            params["analyze_type"] = f"{analyze_type}"

        if page != None:
            params['page'] = f"{page}"

        if size != None:
            params['size'] = f"{size}"

        res = requests.get(self.URL_01, headers=self.headers, params=params)
        document = json.loads(res.text)

        return document
    
    def geo_coord2regioncode(self, x, y, input_coord=None, output_coord=None):
        """
        02 좌표-행정구역정보 변환
        """
        params = {"x": f"{x}",
                  "y": f"{y}"}
        
        if input_coord != None:
            params['input_coord'] = f"{input_coord}"
        
        if output_coord != None:
            params['output_coord'] = f"{output_coord}"
            
        res = requests.get(self.URL_02, headers=self.headers, params=params)
        document = json.loads(res.text)
        
        return document
        
    
    def geo_coord2address(self, x, y, input_coord=None):
        """
        03 좌표-주소 변환
        """
        params = {"x": f"{x}",
                  "y": f"{y}"}
        
        if input_coord != None:
            params['input_coord'] = f"{input_coord}"
            
        res = requests.get(self.URL_03, headers=self.headers, params=params)
        document = json.loads(res.text)
        
        return document
        
    
    def geo_transcoord(self, x, y, output_coord, input_coord=None):
        """
        04 좌표계 변환
        """
        params = {"x": f"{x}",
                  "y": f"{y}",
                  "output_coord": f"{output_coord}"}
        
        if input_coord != None:
            params['input_coord'] = f"{input_coord}"
        
        res = requests.get(self.URL_04, headers=self.headers, params=params)
        document = json.loads(res.text)
        
        return document
        
    
    def search_keyword(self,query,category_group_code=None,x=None,y=None,radius=None,rect=None,page=None,size=None,sort=None):
        """
        05 키워드 검색
        """
        params = {"query": f"{query}"}
        
        if category_group_code != None:
            params['category_group_code'] = f"{category_group_code}"
        if x != None:
            params['x'] = f"{x}"
        if y != None:
            params['y'] = f"{y}"
        if radius != None:
            params['radius'] = f"{radius}"
        if rect != None:
            params['rect'] = f"{rect}"
        if page != None:
            params['page'] = f"{page}"
        if size != None:
            params['size'] = f"{params}"
        if sort != None:
            params['sort'] = f"{sort}"
        
        res = requests.get(self.URL_05, headers=self.headers, params=params)
        document = json.loads(res.text)
        
        return document
    
            
        
    def search_category(self, category_group_code, x, y, radius=None, rect=None, page=None, size=None, sort=None):
        """
        06 카테고리 검색
        """
        params = {'category_group_code': f"{category_group_code}",
                  'x': f"{x}",
                  'y': f"{y}"}
        
        if radius != None:
            params['radius'] = f"{radius}"
        if rect != None:
            params['rect'] = f"{rect}"
        if page != None:
            params['page'] = f"{page}"
        if size != None:
            params['size'] = f"{size}"
        if sort != None:
            params['sort'] = f"{sort}"
            
        res = requests.get(self.URL_06, headers=self.headers, params=params)
        document = json.loads(res.text)
        
        return document

In [4]:
# REST API 키
rest_api_key = "650c8233432728848e448a6d76f3f155"

kakao = KakaoLocalAPI(rest_api_key)

## 편의점들 

In [105]:
places_name = []
road_address_name = []
lat = []
lng = []
distance = []
category_group_name = []

for i in range(len(df)):
  # 편의점
  category_group_code = "CS2"
  x = df['longitude'][i]
  y = df['latitude'][i]
  radius = 300
  ## Request
  result_ = kakao.search_category(category_group_code, x, y, radius)
  # parse
  for k in range(len(result_['documents'])):
    # add in list
    places_name.append(result_['documents'][k]['place_name'])
    road_address_name.append(result_['documents'][k]['road_address_name'])
    lat.append(result_['documents'][k]['x'])
    lng.append(result_['documents'][k]['y'])
    distance.append(result_['documents'][k]['distance'])
    category_group_name.append(result_['documents'][k]['category_group_name'])

In [106]:
conv = pd.DataFrame()

conv['places_name'] = places_name
conv['road_address_name'] = road_address_name
conv['lat'] = lat
conv['lng'] = lng
conv['distance'] = distance
conv['category_group_name'] = category_group_name

conv = conv.drop_duplicates(['road_address_name'], keep = 'first')
conv

,places_name,road_address_name,lat,lng,distance,category_group_name
0,GS25 양평본점,서울 영등포구 양평로30가길 19,126.890408539472,37.542345625392,212,편의점
1,세븐일레븐 양평유성점,서울 영등포구 양평로30나길 14,126.889551547755,37.5433971939308,100,편의점
2,세븐일레븐 양평까뮤스퀘어점,서울 영등포구 양평로30길 14,126.889364685448,37.5412057889037,260,편의점
16,CU 양평한신점,서울 영등포구 양평로24길 9,126.891086345469,37.5407785136294,96,편의점
18,세븐일레븐 양평우림점,서울 영등포구 양평로 149,126.89132789765492,37.538846992992156,208,편의점
...,...,...,...,...,...,...
12361,이마트24 오라카이청계산호텔점,서울 서초구 청계산로9길 1-7,127.054851762929,37.4488232050291,150,편의점
12362,세븐일레븐 서초내곡점,서울 서초구 청계산로 189,127.05550919667,37.4484183428567,170,편의점
12363,이마트24 SMART서초포레스타점,서울 서초구 청계산로11길 7-12,127.05667310274019,37.44826371710339,200,편의점
12364,GS25 청계옛골점,경기 성남시 수정구 청계산로 440,127.071175824544,37.4302968376313,138,편의점


## 음식점들

In [107]:
places_name = []
road_address_name = []
lat = []
lng = []
distance = []
category_group_name = []

for i in range(len(df)):
  # 음식점
  category_group_code = "FD6"
  x = df['longitude'][i]
  y = df['latitude'][i]
  radius = 300
  ## Request
  result_ = kakao.search_category(category_group_code, x, y, radius)
  # parse
  for k in range(len(result_['documents'])):
    # add in list
    places_name.append(result_['documents'][k]['place_name'])
    road_address_name.append(result_['documents'][k]['road_address_name'])
    lat.append(result_['documents'][k]['x'])
    lng.append(result_['documents'][k]['y'])
    distance.append(result_['documents'][k]['distance'])
    category_group_name.append(result_['documents'][k]['category_group_name'])

In [108]:
dinning = pd.DataFrame()

dinning['places_name'] = places_name
dinning['road_address_name'] = road_address_name
dinning['lat'] = lat
dinning['lng'] = lng
dinning['distance'] = distance
dinning['category_group_name'] = category_group_name

dinning = dinning.drop_duplicates(['road_address_name'], keep = 'first')
dinning

,places_name,road_address_name,lat,lng,distance,category_group_name
0,이가네손칼국수,서울 영등포구 양평로28바길 12,126.890882859081,37.5429803680566,223,음식점
1,바우네나주곰탕 선유도점,서울 영등포구 양평로30길 14,126.889364685448,37.5412057889037,260,음식점
3,백미향,서울 영등포구 양평로28바길 10-1,126.891086636445,37.5429120802468,242,음식점
4,풍성 영양돌솥밥전문점,서울 영등포구 양평로28바길 9,126.891059746882,37.5427282516156,245,음식점
5,선진포크,서울 영등포구 양평로30가길 22,126.890774903942,37.5425189566804,231,음식점
...,...,...,...,...,...,...
17302,옛골산장,경기 성남시 수정구 청계산로 440-5,127.0710895856173,37.429904947292854,176,음식점
17303,마포갈비,경기 성남시 수정구 청계산로 441,127.07142927645,37.4306823198422,115,음식점
17304,풍천가 별채,경기 성남시 수정구 청계산로 450,127.072454935165,37.4304258069238,204,음식점
17305,청계가든,경기 성남시 수정구 청계산로449번길 3,127.071763746962,37.430718157922,135,음식점


In [109]:
conv_din = pd.concat([conv, dinning])
conv_din.to_csv('/content/drive/MyDrive/DATA/비저블 맵 프로젝트/conv_din.csv')

## 대형마트

In [110]:
places_name = []
road_address_name = []
lat = []
lng = []
distance = []
category_group_name = []

for i in range(len(df)):
  # 대형마트
  category_group_code = "MT1"
  x = df['longitude'][i]
  y = df['latitude'][i]
  radius = 1000
  ## Request
  result_ = kakao.search_category(category_group_code, x, y, radius)
  # parse
  for k in range(len(result_['documents'])):
    # add in list
    places_name.append(result_['documents'][k]['place_name'])
    road_address_name.append(result_['documents'][k]['road_address_name'])
    lat.append(result_['documents'][k]['x'])
    lng.append(result_['documents'][k]['y'])
    distance.append(result_['documents'][k]['distance'])
    category_group_name.append(result_['documents'][k]['category_group_name'])

In [112]:
MT1 = pd.DataFrame()

MT1['places_name'] = places_name
MT1['road_address_name'] = road_address_name
MT1['lat'] = lat
MT1['lng'] = lng
MT1['distance'] = distance
MT1['category_group_name'] = category_group_name

MT1 = MT1.drop_duplicates(['road_address_name'], keep = 'first')
MT1

,places_name,road_address_name,lat,lng,distance,category_group_name
0,노브랜드 영등포양평점,서울 영등포구 양평로 135,126.89268548684787,37.53818960363232,693,대형마트
1,이마트에브리데이 목동중앙점,서울 양천구 목동동로 411,126.882016986754,37.5366576379574,940,대형마트
12,홈플러스익스프레스 목동점,서울 양천구 목동동로 401,126.880822390803,37.5365230904258,925,대형마트
21,롯데슈퍼 당산역점,서울 영등포구 양평로12길 10-8,126.90161488820415,37.53487989786323,884,대형마트
23,홈플러스익스프레스 당산점,서울 영등포구 당산로44길 3,126.90106107195523,37.53187099393466,734,대형마트
...,...,...,...,...,...,...
4634,코스트코 양재점,서울 서초구 양재대로 159,127.036287876828,37.4619310902256,987,대형마트
4653,홈플러스익스프레스 세곡점,서울 강남구 헌릉로590길 10,127.102960512943,37.4631712453552,394,대형마트
4681,남서울농협하나로마트 우면점,서울 서초구 양재대로2길 116-19,127.015986206099,37.4568718911591,843,대형마트
4709,서울영동농협하나로마트 청계산역점,서울 서초구 청계산로 197,127.0559177383368,37.44766400159238,252,대형마트


In [113]:
conv_to_mart = pd.concat([conv_din, MT1])
conv_to_mart.to_csv('/content/drive/MyDrive/DATA/비저블 맵 프로젝트/conv_to_mart.csv')

## 지하철

In [116]:
places_name = []
road_address_name = []
lat = []
lng = []
distance = []
category_group_name = []

for i in range(len(df)):
  # 지하철
  category_group_code = "SW8"
  x = df['longitude'][i]
  y = df['latitude'][i]
  radius = 500
  ## Request
  result_ = kakao.search_category(category_group_code, x, y, radius)
  # parse
  for k in range(len(result_['documents'])):
    # add in list
    places_name.append(result_['documents'][k]['place_name'])
    road_address_name.append(result_['documents'][k]['road_address_name'])
    lat.append(result_['documents'][k]['x'])
    lng.append(result_['documents'][k]['y'])
    distance.append(result_['documents'][k]['distance'])
    category_group_name.append(result_['documents'][k]['category_group_name'])

,places_name,road_address_name,lat,lng,distance,category_group_name


In [ ]:
SW8 = pd.DataFrame()

SW8['places_name'] = places_name
SW8['road_address_name'] = road_address_name
SW8['lat'] = lat
SW8['lng'] = lng
SW8['distance'] = distance
SW8['category_group_name'] = category_group_name

SW8 = SW8.drop_duplicates(['road_address_name'], keep = 'first')
SW8

In [117]:
# 편의점부터 마트까지 합치기
conv_to_SW8 = pd.concat([conv_to_mart, SW8])
# 저장 
conv_to_SW8.to_csv('/content/drive/MyDrive/DATA/비저블 맵 프로젝트/conv_to_SW8.csv')

### 중간저장

In [115]:
df_ = pd.read_csv('/content/drive/MyDrive/DATA/비저블 맵 프로젝트/conv_to_mart.csv')

,Unnamed: 0,places_name,road_address_name,lat,lng,distance,category_group_name
0,0,GS25 양평본점,서울 영등포구 양평로30가길 19,126.890409,37.542346,212,편의점
1,1,세븐일레븐 양평유성점,서울 영등포구 양평로30나길 14,126.889552,37.543397,100,편의점
2,2,세븐일레븐 양평까뮤스퀘어점,서울 영등포구 양평로30길 14,126.889365,37.541206,260,편의점
3,16,CU 양평한신점,서울 영등포구 양평로24길 9,126.891086,37.540779,96,편의점
4,18,세븐일레븐 양평우림점,서울 영등포구 양평로 149,126.891328,37.538847,208,편의점
...,...,...,...,...,...,...,...
5229,4634,코스트코 양재점,서울 서초구 양재대로 159,127.036288,37.461931,987,대형마트
5230,4653,홈플러스익스프레스 세곡점,서울 강남구 헌릉로590길 10,127.102961,37.463171,394,대형마트
5231,4681,남서울농협하나로마트 우면점,서울 서초구 양재대로2길 116-19,127.015986,37.456872,843,대형마트
5232,4709,서울영동농협하나로마트 청계산역점,서울 서초구 청계산로 197,127.055918,37.447664,252,대형마트


In [ ]:
result_